# Данные

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import h5py
import pandas as pd

with h5py.File('/content/drive/MyDrive/Методы компрессии/common_fields_images.h5') as f:
    images = f['images'][:]
    additional_bits = f['additional_bit'][:]

with open('/content/drive/MyDrive/Методы компрессии/common_fields_labels.txt', encoding='cp1251') as f:
    markup = [e.strip() for e in f.readlines()]

In [ ]:
import numpy as np

def encode_texts(texts):
    def _label_to_num(label, alphabet):
        label_num = []
        for ch in label:
            label_num.append(alphabet.find(ch))
        return np.array(label_num)

    alphabet = ''.join(sorted(set(''.join(texts))))

    nums = np.ones([len(texts), max([len(text) for text in texts])], dtype='int64') * len(alphabet)
    for i, text in enumerate(texts):
        nums[i][:len(text)] = _label_to_num(text, alphabet)

    return nums, alphabet

In [ ]:
labels_encoded, alphabet = encode_texts(markup)

print(alphabet)

images = images.astype('float64') / 255

additional_bits_expanded = np.zeros((len(images), 50, 2))
additional_bits_expanded[:, :, additional_bits] = 1

 -.0123456789АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ


In [ ]:
train_indices = np.random.choice(np.arange(images.shape[0]), int(images.shape[0]*0.8), replace=False)
val_indices = [e for e in np.arange(images.shape[0]) if e not in train_indices]

assert len(set(train_indices) & set(val_indices)) == 0
assert len(set(train_indices) | set(val_indices)) == images.shape[0]

train_imgs = images[train_indices]
val_imgs = images[val_indices]

train_abits = additional_bits_expanded[train_indices]
val_abits = additional_bits_expanded[val_indices]

train_labels = labels_encoded[train_indices]
val_labels = labels_encoded[val_indices]

# Функции

In [ ]:
import tensorflow as tf
from keras import backend as K


class CER(tf.keras.metrics.Metric):
    """
    A custom Keras metric to compute the Character Error Rate
    """
    def __init__(self, name='CER', decode_greedy=True, **kwargs):
        super(CER, self).__init__(name=name, **kwargs)
        self.decode_greedy = decode_greedy
        self.cer_accumulator = self.add_weight(name="total_cer", initializer="zeros")
        self.counter = self.add_weight(name="cer_count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        input_shape = K.shape(y_pred)
        input_length = tf.ones(shape=input_shape[0]) * K.cast(input_shape[1], 'float32')

        decode, log = K.ctc_decode(y_pred, input_length, greedy=True)

        decode = K.ctc_label_dense_to_sparse(decode[0], K.cast(input_length, 'int32'))
        y_true_sparse = K.ctc_label_dense_to_sparse(y_true, K.cast(input_length, 'int32'))
        y_true_sparse = tf.sparse.retain(y_true_sparse, tf.not_equal(y_true_sparse.values, tf.math.reduce_max(y_true_sparse.values)))

        decode = tf.sparse.retain(decode, tf.not_equal(decode.values, -1))
        distance = tf.edit_distance(decode, y_true_sparse, normalize=True)

        self.cer_accumulator.assign_add(tf.reduce_sum(distance))
        self.counter.assign_add(K.cast(len(y_true), 'float32'))

    def result(self):
        return tf.math.divide_no_nan(self.cer_accumulator, self.counter)

    def reset_state(self):
        self.cer_accumulator.assign(0.0)
        self.counter.assign(0.0)


def CTCLoss(y_true, y_pred):
    """
    Compute the training-time loss value
    """
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = K.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

# Модель

In [ ]:
import time

model = tf.keras.models.load_model('/content/drive/MyDrive/Методы компрессии/crnn_common_fields.h5',
                                   custom_objects={'CTCLoss': CTCLoss, 'CER': CER})

# warmup
_ = model.predict([val_abits, val_imgs], verbose=0)

start = time.time()
loss, cer = model.evaluate([val_abits, val_imgs], val_labels)
print(f'Time spent: {round(time.time()-start, 6)}, loss: {round(loss, 6)}, CER: {round(cer, 6)}')

103/103 [==============================] - 6s 49ms/step - loss: 0.1482 - CER: 0.0027
Time spent: 6.911379, loss: 0.148245, CER: 0.002677


# Оптимизация

In [ ]:
!pip install -q tensorflow_model_optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
import tensorflow_model_optimization as tfmot

epochs = 10
batch_size = 256

num_images = train_imgs.shape[0]
end_step = (num_images // 256 + 1) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}


model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4),
              loss=CTCLoss,
              metrics=[CER()])

Первые неудачные эксперименты показали, что дообучаться нужно не слишком мало эпох + возможно не стоит брать слишком большой lr (мы же тут fine-tuning'ом занимаемся).

Вообще, было бы неплохо автоматизировать выбор числа эпох каким-нибудь callback'ом (типа учись не больше условных 20 эпох пока значения фп значительно изменяются каждую эпоху)

In [ ]:
model_for_pruning.fit([train_abits, train_imgs], train_labels, validation_data=([val_abits, val_imgs], val_labels),
                      batch_size=batch_size, epochs=epochs,
                      callbacks=[tfmot.sparsity.keras.UpdatePruningStep(),])

Epoch 1/10
52/52 [==============================] - 54s 702ms/step - loss: 0.1229 - CER: 0.0029 - val_loss: 0.3924 - val_CER: 0.0072
Epoch 2/10
52/52 [==============================] - 39s 759ms/step - loss: 0.1470 - CER: 0.0030 - val_loss: 2.4628 - val_CER: 0.0441
Epoch 3/10
52/52 [==============================] - 33s 631ms/step - loss: 0.2768 - CER: 0.0049 - val_loss: 0.5592 - val_CER: 0.0099
Epoch 4/10
52/52 [==============================] - 33s 631ms/step - loss: 0.2232 - CER: 0.0042 - val_loss: 2.4959 - val_CER: 0.0476
Epoch 5/10
52/52 [==============================] - 32s 618ms/step - loss: 0.3086 - CER: 0.0055 - val_loss: 0.6668 - val_CER: 0.0112
Epoch 6/10
52/52 [==============================] - 34s 652ms/step - loss: 0.3346 - CER: 0.0058 - val_loss: 1.1757 - val_CER: 0.0223
Epoch 7/10
52/52 [==============================] - 34s 656ms/step - loss: 0.4386 - CER: 0.0075 - val_loss: 0.6985 - val_CER: 0.0130
Epoch 8/10
52/52 [==============================] - 34s 649ms/step - 

In [ ]:
start = time.time()
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4),
              loss=CTCLoss,
              metrics=[CER()])
loss, cer = model_for_export.evaluate([val_abits, val_imgs], val_labels)
print(f'Time spent: {round(time.time()-start, 6)}, loss: {round(loss, 6)}, CER: {round(cer, 6)}')

103/103 [==============================] - 7s 57ms/step - loss: 0.7720 - CER: 0.0138
Time spent: 8.591957, loss: 0.772015, CER: 0.013807


Ускорения нет(

In [ ]:
from google.colab import runtime

runtime.unassign()